<a href="https://colab.research.google.com/github/onlygoodman/MLDLpractice/blob/master/project_cfr010.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

"Isolation Forest" 모델 사용
001 feature 골라서 사용하면서 + pca 사용하여 최고점수 도출시도

002 코사인, 피어슨 유사도 특성추가

003 -

004 Isolation Forest가 Robust모델에 비해 f1, auc 스코어 간의 격차가 적어 hyperparameter값 세부조정 시도

- Isolation Forest h-params는 nes=150근처 pca는 사용하지 않을 때 성능이 가장 좋았다. -> 최대한 독립적인 특성만 사용하면서 학습시켰기 때문이라고 생각

005 성능 개선을 위해서는 특성추가가 필요하다고 생각 11, 12, 17에서
2, 4, 9, 10, 11, 12, 16, 17, 18 으로 특성 늘려서 학습

006 4,9번특성이 성능이 잘나와서 4,9,11,12,17,31,32 특성을 사용하고 LOF와 OCS 모델 추가 사용하여 학습

008 코사인유사도, 피어슨유사도, 11,12,17번을 사용하고 robust 모델의 성능이 개선되는 feature을 도출해내는 과정

009 - 008의 실험결과가 14번이 생각이상으로 좋은 특성이었던것 같고 [29, 28, 27, 23, 21, 20, 8, 7, 5, 2]가 모델 성능을 현저히 떨어트리는 특성인것같다. 해당 특성을 제외하고 코사인, 피어슨 유사도를 측정 11,12,14,17,31,32 로 학습한 결과와 비교

010 - 009에서 14번 추가 후 성능상승, 1번부터 다시 최적의 특성탐색



In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from itertools import permutations,combinations

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("darkgrid")


import math #차후 factorial 계산 등 수학적인 계산을 위한 import
from numpy import dot
from numpy.linalg import norm

from scipy.stats import pearsonr
from sklearn.preprocessing import PolynomialFeatures #특성공학을 사용하기 위한 import
from sklearn.preprocessing import StandardScaler #전처리된 데이터들의 scale을 맞춰주기 위한 import
from sklearn.decomposition import PCA #차원 축소 - 주성분의 개수에 따른 최적화 모델 구현을 위한 import



#model
from sklearn import svm
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import KMeans
# from sklearn.cluster import AgglomerativeClustering


from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score #이후 test.csv 파일의 predict 자료의 score을 매기기 위한 import

In [2]:
# Train dataset은 Label이 존재하지 않음
train_df = pd.read_csv("/content/drive/MyDrive/ai7/train.csv")
val_df  = pd.read_csv("/content/drive/MyDrive/ai7/val.csv")
test_df = pd.read_csv("/content/drive/MyDrive/ai7/test.csv")
submit = pd.read_csv("/content/drive/MyDrive/ai7/sample_submission.csv")

In [3]:
clist = list(train_df.columns)
print(clist)

train_data = train_df[clist[1:]].to_numpy()
val_data = val_df[clist[1:]].to_numpy()
val_target = val_df['Class'].to_numpy()

print(val_target)

valcount1=0
valcountm1=0
for i in val_target:
  if i==1:
    valcount1+=1
  else:
    valcountm1+=1

print(valcount1, valcountm1)

outliers_fraction = valcount1/(valcount1+valcountm1)
print("outliers_fraction =",outliers_fraction)

['ID', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30']
[0 0 0 ... 0 0 0]
30 28432
outliers_fraction = 0.0010540369615627855


코사인유사도 구하기

In [4]:
col = [clist[i] for i in range(1, 30+1)] #유사도를 구할 때 사용 할 특성

val_abnor = val_df[val_df['Class']==1][col]
abnor_vector = val_abnor.mean().to_numpy()

def cos_sim(a, b):
  return dot(a, b)/(norm(a)*norm(b))

def cosfeature(data):
  datacs=[]
  for i in data[col].to_numpy():
    datacs.append(cos_sim(abnor_vector, i))
  data['V31']=datacs

cosfeature(train_df)
cosfeature(val_df)
cosfeature(test_df)


피어슨유사도 구하기

In [5]:
def pearson_similarity(a, b):
    return np.dot((a - np.mean(a)), (b - np.mean(b))) / ((np.linalg.norm(a - np.mean(a))) * (np.linalg.norm(b - np.mean(b))))

def pearsonfeature(data):
  datacs=[]
  for i in data[col].to_numpy():
    datacs.append(pearson_similarity(abnor_vector, i))
  data['V32']=datacs

pearsonfeature(train_df)
pearsonfeature(val_df)
pearsonfeature(test_df)

clist = list(train_df.columns)

In [6]:
print(len(clist))
print(clist)

33
['ID', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32']


In [10]:
def predata(data):
  return data.drop(columns=['ID'])[col]

def get_pred_label(model_pred):
    # IsolationForest 모델 출력 (1:정상, -1:불량(사기)) 이므로 (0:정상, 1:불량(사기))로 Label 변환
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

scoreboard={}
maxscoreboard=[]
modelscoreboard={"IF":0, "RC":0, "LOF":0, "OCS":0}
th_score=0 #제출 파일 저장 문턱점수 
maxscore=0 #지난회차 최고점수
testname="010"
poly_dim =1
testnum = 0

mainfeature = [11, 12, 14, 17, 31, 32]

# for k in range(1, 4):
  # for ctu in list(combinations([2,4,9,10,16,18], k)):
      #  for ctu in [i for i in range(1,31) if i not in [11, 12, 17]]:  
for k in [3,15]:
  for ctu in [x for x in range(1, 30+1) if x not in mainfeature+[3,15]]:  
    col = [clist[i] for i in mainfeature+[k]+[ctu]]
    
    train_x = predata(train_df)
    val_x = predata(val_df) # Input Data
    val_y = val_df['Class'] # Label

    test_x = predata(test_df)

    for i in range(len(col), len(col)+1):
      ss = StandardScaler()
      ss.fit(train_x)
      train_x = ss.transform(train_x)
      val_x = ss.transform(val_x)
      test_x = ss.transform(test_x)

      pca = PCA(n_components = i)
      pca.fit(train_x)
      train_pca = pca.transform(train_x)
      val_pca = pca.transform(val_x)
      test_pca = pca.transform(test_x)

      # anomaly_algorithms = [("Robust covariance", EllipticEnvelope(contamination=outliers_fraction)),
      #                     ("One-Class SVM", svm.OneClassSVM(nu=outliers_fraction, kernel="rbf",gamma=0.1)),
      #                     ("Isolation Forest", IsolationForest(max_samples=len(train_pca), contamination=outliers_fraction,random_state=42)),
      #                     ("Local Outlier Factor", LocalOutlierFactor(n_neighbors=35, contamination=outliers_fraction))]

      contamination_controlnumber=0
      
      nes=150

      ##lof hp
      neig=35

      ##ocs hp
      gam=0.1
      ker="rbf"

      model_if = IsolationForest(n_estimators=nes, max_samples=len(train_pca), contamination=outliers_fraction, random_state=42, verbose=0)
      model_ocs = svm.OneClassSVM(nu=outliers_fraction, kernel=ker ,gamma=gam)
      model_rc = EllipticEnvelope(contamination=outliers_fraction + contamination_controlnumber)
      model_lof = LocalOutlierFactor(n_neighbors=neig, contamination=outliers_fraction)

      # ("One-Class SVM",model_ocs) 
      # model_list=[("Isolation Forest", model_if), ("Robust covariance", model_rc),("Local Outlier Factor", model_lof),("One-Class SVM", model_ocs)]
      model_list=[("IF", model_if), ("RC", model_rc)]#,("LOF", model_lof),("OCS", model_ocs)]
      
      for name, m in model_list[1:]:
        
        testnum+=1
        m.fit(train_pca)

        if name == "LOF":
            val_pred = m.fit_predict(val_pca)
        else:
            val_pred = m.predict(val_pca)

        val_pred = get_pred_label(val_pred)
        val_score = f1_score(val_y, val_pred, average='macro')
        val_auc_score = roc_auc_score(val_y, val_pred)
        scoreboard["model:{}, pca-ncom:{}, contamination:{}".format(name, i, outliers_fraction + contamination_controlnumber)]=\
        "f1-score : {}, auc-score : {}".format(round(val_score, 4)*100, round(val_auc_score, 4)*100)

        if val_score >= modelscoreboard[name]-0.0000001:
          maxscore = val_score
          modelscoreboard[name] = val_score
          maxscoreboard.append("no.{}-f1-score:{},\t\t auc-score:{}".format(testnum, val_score, val_auc_score))
          print("----------------------------------------------------")
          print("!!!!!!!!!!!!!!!!!!!!!max score!!!!!!!!!!!!!!!!!!!!!!")
          print("----------------------------------------------------")

          test_pred = m.predict(test_pca) # model prediction  
          test_pred = get_pred_label(test_pred)
          submit['Class'] = test_pred
          submit.to_csv('./submit{}_no.{:0>4}_model_{}.csv'.format(testname, testnum, name), index=False)


        print("no.{:0>4} model_{}\t----------- \tfeature:{},k:{}\t-----------max-{}".format(testnum, name, col, k,maxscore))

        if name == model_list[0][0]:    #is
          print('pca-ncom:{}, n-esti:{}'.format(i, nes),'\n',
              'f1-score : {},\t\tauc-score : {}'.format(round(val_score, 4)*100, round(val_auc_score, 4)*100))
          
        elif name == model_list[1][0]:  #rb
          print('pca-ncom:{}'.format(i),'\n',
              'f1-score : {},\t\tauc-score : {}'.format(round(val_score, 4)*100, round(val_auc_score, 4)*100))
          
        # elif name == model_list[2][0]:  #lof
        #   print('pca-ncom:{}, neig:{}'.format(i, neig),'\n',
        #       'f1-score : {},\t\tauc-score : {}'.format(round(val_score, 4)*100, round(val_auc_score, 4)*100))
          
        # else:                           #ocs
        #   print('pca-ncom:{}, ganma:{}, kenel:{}'.format(i, gam, ker),'\n',
        #       'f1-score : {},\t\tauc-score : {}'.format(round(val_score, 4)*100, round(val_auc_score, 4)*100))
          
        # print(f'Validation F1 Score : [{val_score}]')
        # print(classification_report(val_y, val_pred))
        print()

----------------------------------------------------
!!!!!!!!!!!!!!!!!!!!!max score!!!!!!!!!!!!!!!!!!!!!!
----------------------------------------------------
no.0001 model_RC	----------- 	feature:['V11', 'V12', 'V14', 'V17', 'V31', 'V32', 'V3', 'V1'],k:3	-----------max-0.8156048323280637
pca-ncom:8 
 f1-score : 81.56,		auc-score : 79.97999999999999

----------------------------------------------------
!!!!!!!!!!!!!!!!!!!!!max score!!!!!!!!!!!!!!!!!!!!!!
----------------------------------------------------
no.0002 model_RC	----------- 	feature:['V11', 'V12', 'V14', 'V17', 'V31', 'V32', 'V3', 'V2'],k:3	-----------max-0.8507277214084326
pca-ncom:8 
 f1-score : 85.07000000000001,		auc-score : 83.32000000000001

----------------------------------------------------
!!!!!!!!!!!!!!!!!!!!!max score!!!!!!!!!!!!!!!!!!!!!!
----------------------------------------------------
no.0003 model_RC	----------- 	feature:['V11', 'V12', 'V14', 'V17', 'V31', 'V32', 'V3', 'V4'],k:3	-----------max-0.920973499

In [ ]:
3 [10 13 22]
15 [10, 13, 22, 24, 25, 26, 30]

In [14]:
def predata(data):
  return data.drop(columns=['ID'])[col]

def get_pred_label(model_pred):
    # IsolationForest 모델 출력 (1:정상, -1:불량(사기)) 이므로 (0:정상, 1:불량(사기))로 Label 변환
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

scoreboard={}
maxscoreboard=[]
modelscoreboard={"IF":0, "RC":0, "LOF":0, "OCS":0}
th_score=0 #제출 파일 저장 문턱점수 
maxscore=0 #지난회차 최고점수
testname="010"
poly_dim =1
testnum = 0

mainfeature = [11, 12, 14, 17, 31, 32]

# for k in range(1, 4):
  # for ctu in list(combinations([2,4,9,10,16,18], k)):
      #  for ctu in [i for i in range(1,31) if i not in [11, 12, 17]]:  
for k in [[3,10],[3,13]]:
  for j in [26,30]:
    for ctu in [x for x in range(1, 30+1) if x not in mainfeature+[3,10,13,15,22,26,29,30]]:  
      col = [clist[i] for i in mainfeature+k+[j]+[ctu]]
      
      train_x = predata(train_df)
      val_x = predata(val_df) # Input Data
      val_y = val_df['Class'] # Label

      test_x = predata(test_df)

      for i in range(len(col), len(col)+1):
        ss = StandardScaler()
        ss.fit(train_x)
        train_x = ss.transform(train_x)
        val_x = ss.transform(val_x)
        test_x = ss.transform(test_x)

        pca = PCA(n_components = i)
        pca.fit(train_x)
        train_pca = pca.transform(train_x)
        val_pca = pca.transform(val_x)
        test_pca = pca.transform(test_x)

        # anomaly_algorithms = [("Robust covariance", EllipticEnvelope(contamination=outliers_fraction)),
        #                     ("One-Class SVM", svm.OneClassSVM(nu=outliers_fraction, kernel="rbf",gamma=0.1)),
        #                     ("Isolation Forest", IsolationForest(max_samples=len(train_pca), contamination=outliers_fraction,random_state=42)),
        #                     ("Local Outlier Factor", LocalOutlierFactor(n_neighbors=35, contamination=outliers_fraction))]

        contamination_controlnumber=0
        
        nes=150

        ##lof hp
        neig=35

        ##ocs hp
        gam=0.1
        ker="rbf"

        model_if = IsolationForest(n_estimators=nes, max_samples=len(train_pca), contamination=outliers_fraction, random_state=42, verbose=0)
        model_ocs = svm.OneClassSVM(nu=outliers_fraction, kernel=ker ,gamma=gam)
        model_rc = EllipticEnvelope(contamination=outliers_fraction + contamination_controlnumber)
        model_lof = LocalOutlierFactor(n_neighbors=neig, contamination=outliers_fraction)

        # ("One-Class SVM",model_ocs) 
        # model_list=[("Isolation Forest", model_if), ("Robust covariance", model_rc),("Local Outlier Factor", model_lof),("One-Class SVM", model_ocs)]
        model_list=[("IF", model_if), ("RC", model_rc)]#,("LOF", model_lof),("OCS", model_ocs)]
        
        for name, m in model_list[1:]:
          
          testnum+=1
          m.fit(train_pca)

          if name == "LOF":
              val_pred = m.fit_predict(val_pca)
          else:
              val_pred = m.predict(val_pca)

          val_pred = get_pred_label(val_pred)
          val_score = f1_score(val_y, val_pred, average='macro')
          val_auc_score = roc_auc_score(val_y, val_pred)
          scoreboard["model:{}, pca-ncom:{}, contamination:{}".format(name, i, outliers_fraction + contamination_controlnumber)]=\
          "f1-score : {}, auc-score : {}".format(round(val_score, 4)*100, round(val_auc_score, 4)*100)

          if val_score >= modelscoreboard[name]-0.0000001:
            maxscore = val_score
            modelscoreboard[name] = val_score
            maxscoreboard.append("no.{}-f1-score:{},\t\t auc-score:{}".format(testnum, val_score, val_auc_score))
            print("----------------------------------------------------")
            print("!!!!!!!!!!!!!!!!!!!!!max score!!!!!!!!!!!!!!!!!!!!!!")
            print("----------------------------------------------------")

            # test_pred = m.predict(test_pca) # model prediction  
            # test_pred = get_pred_label(test_pred)
            # submit['Class'] = test_pred
            # submit.to_csv('./submit{}_no.{:0>4}_model_{}.csv'.format(testname, testnum, name), index=False)


          print("no.{:0>4} model_{}\t----------- \tfeature:{},k:{}\t-----------max-{}".format(testnum, name, col, k,maxscore))

          if name == model_list[0][0]:    #is
            print('pca-ncom:{}, n-esti:{}'.format(i, nes),'\n',
                'f1-score : {},\t\tauc-score : {}'.format(round(val_score, 4)*100, round(val_auc_score, 4)*100))
            
          elif name == model_list[1][0]:  #rb
            print('pca-ncom:{}'.format(i),'\n',
                'f1-score : {},\t\tauc-score : {}'.format(round(val_score, 4)*100, round(val_auc_score, 4)*100))
            
          # elif name == model_list[2][0]:  #lof
          #   print('pca-ncom:{}, neig:{}'.format(i, neig),'\n',
          #       'f1-score : {},\t\tauc-score : {}'.format(round(val_score, 4)*100, round(val_auc_score, 4)*100))
            
          # else:                           #ocs
          #   print('pca-ncom:{}, ganma:{}, kenel:{}'.format(i, gam, ker),'\n',
          #       'f1-score : {},\t\tauc-score : {}'.format(round(val_score, 4)*100, round(val_auc_score, 4)*100))
            
          # print(f'Validation F1 Score : [{val_score}]')
          # print(classification_report(val_y, val_pred))
          print()

----------------------------------------------------
!!!!!!!!!!!!!!!!!!!!!max score!!!!!!!!!!!!!!!!!!!!!!
----------------------------------------------------
no.0001 model_RC	----------- 	feature:['V11', 'V12', 'V14', 'V17', 'V31', 'V32', 'V3', 'V10', 'V1'],k:3	-----------max-0.8858506104888013
pca-ncom:9 
 f1-score : 88.59,		auc-score : 86.66

no.0002 model_RC	----------- 	feature:['V11', 'V12', 'V14', 'V17', 'V31', 'V32', 'V3', 'V10', 'V2'],k:3	-----------max-0.8858506104888013
pca-ncom:9 
 f1-score : 87.92,		auc-score : 86.66

----------------------------------------------------
!!!!!!!!!!!!!!!!!!!!!max score!!!!!!!!!!!!!!!!!!!!!!
----------------------------------------------------
no.0003 model_RC	----------- 	feature:['V11', 'V12', 'V14', 'V17', 'V31', 'V32', 'V3', 'V10', 'V4'],k:3	-----------max-0.8964462129361583
pca-ncom:9 
 f1-score : 89.64,		auc-score : 88.32

----------------------------------------------------
!!!!!!!!!!!!!!!!!!!!!max score!!!!!!!!!!!!!!!!!!!!!!
---------

In [ ]:
3 10 [9, 25, 26, 30]
3 13 [26, 30]
15 13 [30]
15 22 []